In [1]:
import random


# Constants
class_timings = ["8:30-9:50", "10:00-11:20", "11:30-12:50", "1:00-2:20", "2:30-5:20"]
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "None"]
valid_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
sections = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
courses = ["AI", "PDC", "SE", "SDA", "CNET", "LA", "NC", "OS", "DB"]
professors = ["Osama", "Aleem", "Majid", "Imran", "Shujaat", "Ali", "Zain", "Owais", "Sara", "Noor", "Maaz", "Hassan"]
clash_types = ["None", "same professor", "same room", "strength", "professor", "same timeslot", "less strength", "day2", "course", "section"]
rooms = ['Room 1', 'Room 2', 'Room 3', 'Room 4', 'Room 5', 'Hall 1', 'Hall 2', 'Hall 3', 'Hall 4', 'Hall 5']
max_courses_per_section = 5
max_courses_per_professor = 3
course_type = ["Theory", "Lab"]
class_strengths = [50, 100]


NUM_SECTIONS = len(sections)
print(NUM_SECTIONS)
NUM_VALID_DAYS = len(valid_days)
NUM_COURSES = len(courses)
NUM_ROOMS = len(rooms)
NUM_PROFESSORS = len(professors)
DAYS_IN_WEEK = len(days_of_week)
SLOTS_PER_DAY = len(class_timings)
MORNING_SLOTS = SLOTS_PER_DAY - 1
AFTERNOON_SLOTS = 1

class Chromosome:
    def __init__(self):
        self.schedule = []



    def decode_chromosome(self):
        decoded_schedule = []
        for entry in self.schedule:
            course_index = int(entry[0], 2)
            course_type_index = int(entry[1], 2)
            section_index = int(entry[2], 2)
            strength_index = int(entry[3], 2)
            professor_index = int(entry[4], 2)
            room_index = int(entry[5], 2)
            slot_index = int(entry[6], 2)
            day1_index = int(entry[7], 2)
            day2_index = int(entry[8], 2)
            clash_index = int(entry[9], 2)

            course = courses[course_index]
            type_ = course_type[course_type_index]
            section = sections[section_index]
            classroom_strength = class_strengths[strength_index]
            professor = professors[professor_index]
            room = rooms[room_index]
            timeslot = class_timings[slot_index]
            day1 = days_of_week[day1_index]
            day2 = days_of_week[day2_index]
            clash_type = clash_types[clash_index]

            decoded_entry = (course, type_, section, classroom_strength, professor, room, timeslot, day1, day2, clash_type)
            decoded_schedule.append(decoded_entry)

        return decoded_schedule

    def encode_chromosome(self, course, type_, section, classroom_strength, professor, room, timeslot, day1, day2, clash_type):
        course_index = courses.index(course)
        course_type_index = course_type.index(type_)
        section_index = sections.index(section)
        strength_index = class_strengths.index(classroom_strength)
        professor_index = professors.index(professor)
        room_index = rooms.index(room)
        day1_index = days_of_week.index(day1)

        # Handle day2 separately
        if day2 != "None":
            day2_index = days_of_week.index(day2)
        else:
            day2_index = len(days_of_week) - 1  # Set day2 index to the last index in the list for "None"

        clash_index = clash_types.index(clash_type)

        chromosome = [
            format(course_index, '0' + str(len(bin(len(courses) - 1)) - 2) + 'b'),
            format(course_type_index, '0' + str(len(bin(len(course_type) - 1)) - 2) + 'b'),
            format(section_index, '0' + str(len(bin(len(sections) - 1)) - 2) + 'b'),
            format(strength_index, '0' + str(len(bin(len(class_strengths) - 1)) - 2) + 'b'),
            format(professor_index, '0' + str(len(bin(len(professors) - 1)) - 2) + 'b'),
            format(room_index, '0' + str(len(bin(len(rooms) - 1)) - 2) + 'b'),
            format(timeslot, '0' + str(len(bin(len(class_timings) - 1)) - 2) + 'b'),
            format(day1_index, '0' + str(len(bin(len(days_of_week) - 1)) - 2) + 'b'),
            format(day2_index, '0' + str(len(bin(len(days_of_week) - 1)) - 2) + 'b'),
            format(clash_index, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')
        ]
        return chromosome


    def initialize(self):
        if not professors:
            raise ValueError("No professors available.")

        # Initialize counters for professors and sections
        professor_counts = {professor: 0 for professor in professors}
        section_courses = {section: [] for section in sections}

        # Assign courses to sections
        for section in sections:
            available_courses = courses.copy()
            for i in range(max_courses_per_section):
                if not available_courses:
                    break

                if i < 2:
                    type_ = course_type[0]
                    day1 = days_of_week[0]
                    day2 = random.choice(valid_days)
                elif i < 4:
                    type_ = course_type[0]
                    day1 = days_of_week[1]
                    day2 = random.choice(valid_days)
                else:
                    type_ = course_type[1]
                    day1 = random.choice(valid_days)
                    day2 = "None"

                course = random.choice(courses)
                professor = random.choice(professors)
                room = random.choice(rooms)
                if type_ == "Lab":
                    timeslot = len(class_timings) - 1  # Assign Lab to the last slot
                else:
                    timeslot = random.randint(0, len(class_timings) - 2)  # Random slot for Theory courses

                # Initialize strength of the classroom
                classroom_strength = random.choice(class_strengths)
                clash_type = "None"
                section = random.choice(sections)
                # Update counts
                professor_counts[professor] += 1
                #section_courses[section].append(course)

                chromosome = self.encode_chromosome(course, type_, section, classroom_strength, professor, room, timeslot, day1, day2, clash_type)
                self.schedule.append(chromosome)




    def print_fitness_only(self):
        decoded_schedule = self.decode_chromosome()
        fitness = self.calculate_fitness(decoded_schedule)
        print(fitness)


    def print_fitness(self):
        decoded_schedule = self.decode_chromosome()

        # Sort decoded schedule entries based on day1 and timeslot
        sorted_decoded_schedule = sorted(decoded_schedule, key=lambda x: (valid_days.index(x[7]), class_timings.index(x[6])))

        print("Course\tType\tSection\tStrength\tProfessor\tRoom\tTime\t\tDay 1\t\tDay 2\t\tClash")
        for entry in sorted_decoded_schedule:
            course, type_, section, classroom_strength, professor, room, timeslot, day1, day2, clash_type = entry
            print(f"{course}\t{type_}\t{section}\t{classroom_strength}\t\t{professor}\t\t{room}\t{timeslot}\t{day1}\t\t{day2}\t{clash_type}")

        fitness = self.calculate_fitness(sorted_decoded_schedule)
        print(fitness)




    def mutate(self):
        mutation_rate = 0.1  # Define mutation rate inside the function

        # Define indices for professor_index, room_index, and timeslot
        indices_to_mutate = [0, 2, 4, 5, 6, 8]

        for i, chromosome in enumerate(self.schedule):
            if chromosome[9] != '0000' and chromosome[9] != '0':
                new_chromosome = list(chromosome)  # Convert tuple to list to allow modification
                clash_index = int(chromosome[9], 2)

                # Iterate through the indices to potentially mutate
                for index in indices_to_mutate:
                    if index == 4 and (clash_index == 1 or clash_index == 4):
                        array_length = NUM_PROFESSORS
                        new_value = random.randint(0, array_length - 1)
                        new_chromosome[index] = format(new_value, '0' + str(len(bin(len(professors) - 1)) - 2) + 'b')
                        new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')

                    elif index == 5 and (clash_index == 2 or clash_index == 3 or clash_index == 6):
                        array_length = NUM_ROOMS
                        new_value = random.randint(0, array_length - 1)
                        new_chromosome[index] = format(new_value, '0' + str(len(bin(len(rooms) - 1)) - 2) + 'b')
                        new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')

                    elif index == 6 and (clash_index == 1 or clash_index == 2 or clash_index == 5):
                        course_type_index = int(new_chromosome[1], 2)
                        if course_type_index != 1:
                           array_length = MORNING_SLOTS
                           new_value = random.randint(0, array_length - 1)
                           new_chromosome[index] = format(new_value, '0' + str(len(bin(len(class_timings) - 1)) - 2) + 'b')
                           new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')
                        else:
                           array_length = NUM_SECTIONS
                           new_value = random.randint(0, array_length - 1)
                           new_chromosome[2] = format(new_value, '0' + str(len(bin(len(sections) - 1)) - 2) + 'b')
                           new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')

                    elif index == 0 and clash_index == 8:
                        array_length = NUM_COURSES
                        new_value = random.randint(0, array_length - 1)
                        new_chromosome[index] = format(new_value, '0' + str(len(bin(len(courses) - 1)) - 2) + 'b')
                        new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')

                    elif index == 8 and clash_index == 7:
                        array_length = NUM_VALID_DAYS
                        new_value = random.randint(0, array_length - 1)
                        new_chromosome[index] = format(new_value, '0' + str(len(bin(len(days_of_week) - 1)) - 2) + 'b')
                        new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')

                    elif index == 2 and clash_index == 9:
                        array_length = NUM_SECTIONS
                        new_value = random.randint(0, array_length - 1)
                        new_chromosome[index] = format(new_value, '0' + str(len(bin(len(sections) - 1)) - 2) + 'b')
                        new_chromosome[9] = format(0, '0' + str(len(bin(len(clash_types) - 1)) - 2) + 'b')

                self.schedule[i] = tuple(new_chromosome)  # Convert back to tuple and assign

                # Debugging
                #print(new_chromosome)



    def calculate_fitness_only(self, decoded_schedule):
        clashes = 0
        professors_slots = {professor: set() for professor in professors}
        professors_unique_courses = {professor: set() for professor in professors}  # Dictionary to store unique courses assigned to each professor
        schedule_info = {}  # Dictionary to store schedule information for each room and timeslot
        section_courses_count = {section: 0 for section in sections}  # Dictionary to store count of courses assigned to each section

        for index, entry in enumerate(decoded_schedule):
            course, type_, section, classroom_strength, professor, room, timeslot, day1, day2, clash_type = entry


            if day2 != "None" and (days_of_week.index(day2) != (days_of_week.index(day1) + 2)) or (day1 == day2):
                clashes += 1
                clash_type = clash_types[7]
                #print(f"Clash: Lecture for course {course} in section {section} is scheduled on adjacent days {day1} and {day2}.")


            # Update professors_unique_courses
            professors_unique_courses[professor].add(course)
            # Check if the professor is assigned more than 3 unique courses
            if len(professors_unique_courses[professor]) > 3:
                clashes += 1
                clash_type = clash_types[4]  # Set clash type to "professor"

            # Check if the classroom strength exceeds 60 and the room starts with 'R'
            if classroom_strength > 60 and room.startswith('R'):
                clashes += 1
                clash_type = clash_types[3]

            # Check if the classroom strength is less than 60 and the room starts with 'H'
            if classroom_strength < 60 and room.startswith('H'):
                clashes += 1
                clash_type = clash_types[6]


            # Check if the room is already assigned to another section at this timeslot and day
            if (room, timeslot, day1) in schedule_info and schedule_info[(room, timeslot, day1)] != section:
                clashes += 1
                clash_type = clash_types[2]
            else:
                schedule_info[(room, timeslot, day1)] = section


            # Check if the professor is already assigned to another course at this timeslot and day
            if (professor, timeslot, day1) in professors_slots:
                clashes += 1
                clash_type = clash_types[1]
            else:
                professors_slots[(professor, timeslot, day1)] = 1

            # Check for conflicts in timeslots for courses in the same section
            for other_entry in decoded_schedule[index + 1:]:
                other_course, other_type_, other_section, other_classroom_strength, other_professor, other_room, other_timeslot, other_day1, _, _ = other_entry
                if section == other_section and day1 == other_day1 and timeslot == other_timeslot:
                    clashes += 1
                    clash_type = clash_types[5]
                # Check if the section is taking two same courses
                if section == other_section and course == other_course:
                    clashes += 1
                    clash_type = clash_types[8]

            # Update section_courses_count
            section_courses_count[section] += 1
            # Check if section is assigned to more than 5 courses
            if section_courses_count[section] > 5:
                clashes += 1
                clash_type = clash_types[9]


            # Update the entry with clash type
            slot_index = class_timings.index(timeslot)
            encoded_entry = self.encode_chromosome(course, type_, section, classroom_strength, professor, room, slot_index, day1, day2, clash_type)
            self.schedule[index] = encoded_entry

        # Calculate fitness by subtracting clashes from a perfect score
        fitness = 100 - clashes
        return fitness



    def calculate_fitness(self, decoded_schedule):
        clashes = 0
        professors_slots = {professor: set() for professor in professors}
        professors_unique_courses = {professor: set() for professor in professors}  # Dictionary to store unique courses assigned to each professor
        schedule_info = {}  # Dictionary to store schedule information for each room and timeslot
        section_courses_count = {section: 0 for section in sections}  # Dictionary to store count of courses assigned to each section

        for index, entry in enumerate(decoded_schedule):
            course, type_, section, classroom_strength, professor, room, timeslot, day1, day2, clash_type = entry


            if day2 != "None" and (days_of_week.index(day2) != (days_of_week.index(day1) + 2)) or (day1 == day2):
                clashes += 1
                #clash_type = clash_types[7]
                print(f"Clash: Lecture for course {course} in section {section} is scheduled on adjacent days {day1} and {day2}.")

            # Update professors_unique_courses
            professors_unique_courses[professor].add(course)
            # Check if the professor is assigned more than 3 unique courses
            if len(professors_unique_courses[professor]) > 3:
                clashes += 1
                #clash_type = clash_types[4]  # Set clash type to "professor"
                print(f"Clash: Professor {professor} has more than 3 courses.")

            # Check if the classroom strength exceeds 60 and the room starts with 'R'
            if classroom_strength > 60 and room.startswith('R'):
                clashes += 1
                #clash_type = clash_types[3]
                print(f"Clash: Section {section}: Room {room} assigned to course {course} has insufficient capacity for classroom strength {classroom_strength}.")

            # Check if the professor is already assigned to another course at this timeslot and day
            if (professor, timeslot, day1) in professors_slots:
                clashes += 1
                #clash_type = clash_types[1]
                print(f"Clash: Section {section}: Professor {professor} is already assigned to another course at timeslot {timeslot} on {day1}.")
            else:
                professors_slots[(professor, timeslot, day1)] = 1



            # Check if the room is already assigned to another section at this timeslot and day
            if (room, timeslot, day1) in schedule_info and schedule_info[(room, timeslot, day1)] != section:
                clashes += 1
               #clash_type = clash_types[2]
                print(f"Clash: Section {section} is already assigned to another room at timeslot {timeslot} on {day1}.")
            else:
                schedule_info[(room, timeslot, day1)] = section

            # Check for conflicts in timeslots for courses in the same section
            for other_entry in decoded_schedule[index + 1:]:
                other_course, other_type_, other_section, other_classroom_strength, other_professor, other_room, other_timeslot, other_day1, _, _ = other_entry
                if section == other_section and day1 == other_day1 and timeslot == other_timeslot:
                    clashes += 1
                    #clash_type = clash_types[5]
                    print(f"Clash: Course {course} in section {section} is scheduled at the same timeslot {timeslot} on {day1} as course {other_course}.")
                # Check if the section is taking two same courses
                if section == other_section and course == other_course:
                    clashes += 1
                    #clash_type = clash_types[8]
                    print(f"Clash: Course {course} is repeating for section {section}.")


            # Update section_courses_count
            section_courses_count[section] += 1
            # Check if section is assigned to more than 5 courses
            if section_courses_count[section] > 5:
                clashes += 1
                #clash_type = clash_types[9]
                print(f"Clash: Section {section} has more than 5 courses.")


            # Update the entry with clash type
            slot_index = class_timings.index(timeslot)
            encoded_entry = self.encode_chromosome(course, type_, section, classroom_strength, professor, room, slot_index, day1, day2, clash_type)
            self.schedule[index] = encoded_entry

        # Calculate fitness by subtracting clashes from a perfect score
        #print("professors_slots: ", professors_slots)
        fitness = 100 - clashes
        return fitness




def generate_population(population_size):
    population = []
    for _ in range(population_size):
        chromosome = Chromosome()
        chromosome.initialize()
        population.append(chromosome)
    return population



def select_parents(population):
    # Select parents based on their fitness scores
    decoded_population = [individual.decode_chromosome() for individual in population]
    fitness_scores = [individual.calculate_fitness_only(decoded_chromosome) for decoded_chromosome, individual in zip(decoded_population, population)]
    sorted_population = sorted(zip(population, fitness_scores), key=lambda x: x[1], reverse=True)
    parent1, parent2 = sorted_population[0][0], sorted_population[1][0]
    return parent1, parent2


def crossover(parent1, parent2):
    # Perform crossover by swapping a portion of their chromosomes
    crossover_point = random.randint(0, len(parent1.schedule) - 1)
    child1_schedule = parent1.schedule[:crossover_point] + parent2.schedule[crossover_point:]
    child2_schedule = parent2.schedule[:crossover_point] + parent1.schedule[crossover_point:]
    child1 = Chromosome()
    child2 = Chromosome()
    child1.schedule = child1_schedule
    child2.schedule = child2_schedule
    return child1, child2


def genetic_algorithm(population_size, generations):
    population = generate_population(population_size)

    #print("Initial Population:")
    #for idx, chromosome in enumerate(population):
    #    print(f"Chromosome {idx + 1}:")
    #    chromosome.print_fitness()
    #    #chromosome.print_fitness()
    #    print()

    for generation in range(generations):
        print(f"Generation {generation + 1}:")
        parent1, parent2 = select_parents(population)
        print("Selected parents:")
        p1_decoded = parent1.decode_chromosome()
        p2_decoded = parent2.decode_chromosome()
        p1_fitness = parent1.calculate_fitness_only(p1_decoded)
        p2_fitness = parent2.calculate_fitness_only(p2_decoded)
        print(p1_fitness)
        print(p2_fitness)

        if(p1_fitness == 100):
            break

        print("Performing crossover...")
        child1, child2 = crossover(parent1, parent2)
        print("Children after crossover:")
        c1_decoded = child1.decode_chromosome()
        c2_decoded = child2.decode_chromosome()
        c1_fitness = child1.calculate_fitness_only(c1_decoded)
        c1_fitness = child2.calculate_fitness_only(c2_decoded)

        print("Fitness after Performing mutation on Child 1...")
        child1.mutate()
        c1_mutated_decoded = child1.decode_chromosome()
        c1_fitness_after_mutation = child1.calculate_fitness_only(c1_mutated_decoded)
        print(c1_fitness_after_mutation)
        print("Fitness after Performing mutation on Child 2...")
        child2.mutate()
        c2_mutated_decoded = child2.decode_chromosome()
        c2_fitness_after_mutation = child2.calculate_fitness_only(c2_mutated_decoded)
        print(c2_fitness_after_mutation)

        # Replace the least fit individuals in the population with the children
        # Decode chromosomes for the entire population
        decoded_population = [individual.decode_chromosome() for individual in population]
        # Calculate fitness scores for each individual in the population
        fitness_scores = [individual.calculate_fitness_only(decoded_chromosome) for decoded_chromosome, individual in zip(decoded_population, population)]
        # Combine individuals with their fitness scores
        population_with_fitness = list(zip(population, fitness_scores))
        # Sort the population based on fitness scores
        sorted_population = sorted(population_with_fitness, key=lambda x: x[1])
        # Replace the least fit individuals with the children
        population[population.index(sorted_population[0][0])] = child1
        population[population.index(sorted_population[1][0])] = child2


        #print("Population after replacement:")
        #for idx, chromosome in enumerate(population):
        #   print(f"Chromosome {idx + 1}:")
        #    chromosome.print_fitness()
        #    print()
        print()
    parent1.print_fitness()


if __name__ == "__main__":
    population_size = 100  # Adjust population size as needed
    generations = 100  # Number of generations to run
    genetic_algorithm(population_size, generations)


10
Generation 1:
Selected parents:
8
0
Performing crossover...
Children after crossover:
Fitness after Performing mutation on Child 1...
11
Fitness after Performing mutation on Child 2...
2

Generation 2:
Selected parents:
11
8
Performing crossover...
Children after crossover:
Fitness after Performing mutation on Child 1...
29
Fitness after Performing mutation on Child 2...
24

Generation 3:
Selected parents:
29
24
Performing crossover...
Children after crossover:
Fitness after Performing mutation on Child 1...
39
Fitness after Performing mutation on Child 2...
32

Generation 4:
Selected parents:
39
32
Performing crossover...
Children after crossover:
Fitness after Performing mutation on Child 1...
32
Fitness after Performing mutation on Child 2...
36

Generation 5:
Selected parents:
39
36
Performing crossover...
Children after crossover:
Fitness after Performing mutation on Child 1...
41
Fitness after Performing mutation on Child 2...
36

Generation 6:
Selected parents:
41
39
Performi